# 65536
## Problem Statement
Simulating a game which is similar to 2048 but instead of terminating the game when a tile becomes 2048, we continue to achieve a maximum tile possible or get a tile of value 65536


In [2]:
%pip install pygame

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pygame
import random
import copy
import time
import math

# Define some constants for the game
GRID_SIZE = 4
TILE_SIZE = 100
WINDOW_SIZE = TILE_SIZE * GRID_SIZE
BG_COLOR = (187, 173, 160)
TILE_COLORS = {
    0: (205, 193, 180),
    2: (238, 228, 218),
    4: (237, 224, 200),
    8: (242, 177, 121),
    16: (245, 149, 99),
    32: (246, 124, 95),
    64: (246, 94, 59),
    128: (237, 207, 114),
    256: (237, 204, 97),
    512: (237, 200, 80),
    1024: (237, 197, 63),
    2048: (237, 194, 46)
}

# utility functions

def get_direction_text(direction):
    return "up" if direction == 0 else "right" if direction == 1 else "down" if direction == 2 else "left" if direction == 3 else direction

pygame 2.5.2 (SDL 2.28.3, Python 3.11.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


# 2048 Simulation Project Documentation

## Features

### 1. Game Initialization
- The `Game` class is designed to simulate the 2048 game.
- It includes an optional graphical user interface (GUI) using the Pygame library.
- Users can specify whether to render the GUI for visual representation or run in a non-GUI mode for testing purposes.

### 2. Game Board
- The game board is represented as a grid with customizable dimensions (`GRID_SIZE`) = 4.
- 2 tiles are initialized with random values (2 or 4) upon starting the game.

### 3. User Interface (GUI)
- If the GUI is enabled, the simulation renders the game window using Pygame.
- Tiles are displayed with appropriate colors, and their values are shown in the center.
- A scoring system is implemented to keep track of the user's progress.

### 4. Tile Movement
- Users can move tiles in four directions: up, down, left, and right.
- Tiles slide as far as possible in the specified direction, merging with adjacent tiles of the same value.

### 5. Random Tile Generation
- New tiles are added to the grid randomly, with a 90% probability of a 2 and a 10% probability of a 4.
- This randomness introduces a dynamic element to the game.

### 6. Scoring
- The score is calculated based on the values of merged tiles during movements.
- The goal is to achieve the highest score possible through strategic tile merging.

### 7. Game Over Detection
- The simulation checks for game-over conditions after each move.
- Game over is determined if there are no empty tiles or if no adjacent tiles have the same value.
- In GUI mode, a "Game Over" message is displayed on the window when the game concludes.

## Usage

### Instantiation
```python
game = Game(gui=True)  # Creates a new game with GUI
# OR
game = Game(gui=False)  # Creates a new game without GUI
```

### Game Actions
- `game.move(direction)`: Performs a move in the specified direction (0: up, 1: right, 2: down, 3: left).
- `game.reset()`: Resets the game board to its initial state.
- `game.check_game_over()`: Checks if the game is over.

### Example
```python
game = Game()
game.move(1)  # Move tiles to the right
game.render()  # Render the updated game state
```

### Notes
- The `__str__` method provides a string representation of the current game state.

## Dependencies
- Pygame library (for GUI): Install via `pip install pygame`.


In [2]:
class Game:
    def __init__(self, gui=True, grid=None):
        # the gui flag is used to determine whether to render the game or not (might be useful for testing)
        # Initialize the game
        if gui:
            self.gui = True
            pygame.init()
            self.window = pygame.display.set_mode((WINDOW_SIZE, WINDOW_SIZE))
            self.font = pygame.font.SysFont("Arial", 32)
        if not gui:
            self.gui = False
        self.score = 0
        if grid:
            self.grid = grid
        else:
            self.reset()

    def reset(self):
        self.grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        self.add_random_tile()
        self.add_random_tile()
        self.score = 0

    def add_random_tile(self):
        # Add a random tile to the grid, probability of adding a 2 is 90% and 4 is 10%
        empty_tiles = []
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 0:
                    empty_tiles.append((i, j))
        if empty_tiles:
            i, j = random.choice(empty_tiles)
            self.grid[i][j] = 2 if random.random() < 0.9 else 4

    def render(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                # draw the tile
                pygame.draw.rect(self.window, TILE_COLORS[self.grid[i][j]], (j * TILE_SIZE, i * TILE_SIZE, TILE_SIZE, TILE_SIZE))

                # add the text to the tile
                text = self.font.render(str(self.grid[i][j]) if self.grid[i][j] else "", True, (119, 110, 101))
                text_rect = text.get_rect()
                text_rect.center = (j * TILE_SIZE + TILE_SIZE / 2, i * TILE_SIZE + TILE_SIZE / 2)
                self.window.blit(text, text_rect)

                # add a border around the tiles
                pygame.draw.rect(self.window, (187, 173, 160), (j * TILE_SIZE, i * TILE_SIZE, TILE_SIZE, TILE_SIZE), 5)

        pygame.display.update()

    def move(self, direction):
        # direction: 0 - up, 1 - right, 2 - down, 3 - left
        # slide the tiles in the given direction
        merged = [[False for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
        initial_grid = [row[:] for row in self.grid]

        # Slide tiles as far as possible in the given direction, merging tiles of the same value once.
        if direction == 0:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(i): # k checks all tiles between i and 0
                        if self.grid[k][j] == 0:
                            shift += 1
                    
                    if shift:
                        self.grid[i - shift][j] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if i - shift - 1 >= 0 and self.grid[i - shift - 1][j] == self.grid[i - shift][j] and not merged[i - shift - 1][j] and not merged[i - shift][j]:
                        self.score += self.grid[i - shift][j] * 2
                        self.grid[i - shift - 1][j] *= 2
                        self.grid[i - shift][j] = 0
                        merged[i - shift - 1][j] = True

        elif direction == 1:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE - 1, -1, -1):
                    shift = 0
                    for k in range(j + 1, GRID_SIZE):
                        if self.grid[i][k] == 0:
                            shift += 1

                    if shift:
                        self.grid[i][j + shift] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if j + shift + 1 < GRID_SIZE and self.grid[i][j + shift + 1] == self.grid[i][j + shift] and not merged[i][j + shift + 1] and not merged[i][j + shift]:
                        self.score += self.grid[i][j + shift] * 2
                        self.grid[i][j + shift + 1] *= 2
                        self.grid[i][j + shift] = 0
                        merged[i][j + shift + 1] = True

        elif direction == 2:
            for i in range(GRID_SIZE - 1, -1, -1):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(i + 1, GRID_SIZE):
                        if self.grid[k][j] == 0:
                            shift += 1

                    if shift:
                        self.grid[i + shift][j] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if i + shift + 1 < GRID_SIZE and self.grid[i + shift + 1][j] == self.grid[i + shift][j] and not merged[i + shift + 1][j] and not merged[i + shift][j]:
                        self.score += self.grid[i + shift][j] * 2
                        self.grid[i + shift + 1][j] *= 2
                        self.grid[i + shift][j] = 0
                        merged[i + shift + 1][j] = True

        elif direction == 3:
            for i in range(GRID_SIZE):
                for j in range(GRID_SIZE):
                    shift = 0
                    for k in range(j):
                        if self.grid[i][k] == 0:
                            shift += 1

                    if shift:
                        self.grid[i][j - shift] = self.grid[i][j]
                        self.grid[i][j] = 0

                    if j - shift - 1 >= 0 and self.grid[i][j - shift - 1] == self.grid[i][j - shift] and not merged[i][j - shift - 1] and not merged[i][j - shift]:
                        self.score += self.grid[i][j - shift] * 2
                        self.grid[i][j - shift - 1] *= 2
                        self.grid[i][j - shift] = 0
                        merged[i][j - shift - 1] = True

        if self.grid != initial_grid:
            self.add_random_tile()
            if self.check_game_over():
                if self.gui:
                    pygame.time.delay(5000)
                    pygame.quit()
                return (1, self.grid)
            if self.gui:
                self.render()
            return (0, self.grid)
        else:
            return (2, self.grid)

    def check_game_over(self):
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                if self.grid[i][j] == 0:
                    return
                if i > 0 and self.grid[i - 1][j] == self.grid[i][j]:
                    return
                if i < GRID_SIZE - 1 and self.grid[i + 1][j] == self.grid[i][j]:
                    return
                if j > 0 and self.grid[i][j - 1] == self.grid[i][j]:
                    return
                if j < GRID_SIZE - 1 and self.grid[i][j + 1] == self.grid[i][j]:
                    return

        # overlay text on the screen
        if self.gui:
            text = self.font.render("Game Over!", True, (119, 110, 101))
            text_rect = text.get_rect()
            text_rect.center = (WINDOW_SIZE / 2, WINDOW_SIZE / 2)
            self.window.blit(text, text_rect)
            pygame.display.update()
        return True
        

    def __str__(self):
        return "\n".join([" ".join([str(self.grid[i][j]) for j in range(GRID_SIZE)]) for i in range(GRID_SIZE)])

### Run the cell below to play the game manually

Game object uses pygame to display and in the background it uses the class to work

It has a move function which takes the direction as input and moves the tiles in that direction
Possible directions are: 'up', 'down', 'left', 'right' which are represented by the 0, 2, 3, 1 respectively




In [4]:
game = Game()
game.render()

# close the game window when the user presses the close button
done = False
while not done:
    # game loop
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            print("Exiting...")
            done = True
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_r:
                game.reset()
                game.render()
            elif event.key == pygame.K_UP:
                if game.move(0)[0] == 1:
                    done = True
            elif event.key == pygame.K_RIGHT:
                if game.move(1)[0] == 1:
                    done = True
            elif event.key == pygame.K_DOWN:
                if game.move(2)[0] == 1:
                    done = True
            elif event.key == pygame.K_LEFT:
                if game.move(3)[0] == 1:
                    done = True
        else:
            continue

Exiting...


### Random moves simulation

Done to demonstrate the game is working properly


In [ ]:
game = Game()
game.render()

done = False
while not done:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.display.quit()
            pygame.quit()
            print("Exiting...")
            done = True
    pygame.time.delay(200)
    if game.move(random.randint(0, 3))[0] == 1:
        done = True
        

### Monte Carlo Tree Search (MCTS)
**Monte Carlo Tree Search (MCTS) Overview**

Monte Carlo Tree Search (MCTS) is a popular algorithm used in decision-making processes, particularly in artificial intelligence for games and optimization problems. It is commonly employed when the full state space is too vast to explore exhaustively.

### Key Concepts:

1. **Tree Search:**
   - MCTS builds a tree structure representing possible sequences of moves in a game or actions in a decision-making scenario.

2. **Monte Carlo Simulation:**
   - The algorithm relies on random sampling (Monte Carlo simulations) to explore parts of the decision space, gradually refining its understanding of the most promising choices.

3. **Four Key Steps:**
   - **Selection:** Starting from the root of the tree, traverse down the tree based on certain criteria (often using UCT - Upper Confidence Bounds for Trees) to find a promising node.
   - **Expansion:** Expand the selected node by adding one or more child nodes representing possible moves or actions.
   - **Simulation:** Conduct a Monte Carlo simulation from the newly added node. This involves making random moves or decisions until a terminal state is reached.
   - **Backpropagation:** Update the statistics (e.g., visit count, total reward) of the nodes visited during the simulation. Propagate this information up the tree.

### MCTS in the Context of 2048 Solver:

- **Selection:** The algorithm selects moves (up, down, left, right) based on accumulated statistics, favoring moves that lead to higher scores in the simulations.
  
- **Expansion:** For each selected move, the algorithm explores possible outcomes by adding child nodes representing different game states.

- **Simulation:** The `random_policy` function simulates multiple random games from each newly added node to estimate the potential outcomes of the move.

- **Backpropagation:** The results of the simulations are used to update the statistics of the nodes visited during the selection and expansion steps.

### Purpose in 2048 Solver:

- **Optimal Move Selection:** MCTS helps the AI make informed decisions on the best move in a given game state.
  
- **Dynamic Decision Making:** By using random simulations, the algorithm adapts to the dynamic nature of the game, providing a balance between exploration and exploitation.

- **Efficient Exploration:** Instead of exhaustively searching the entire decision space, MCTS focuses on promising regions, making it suitable for scenarios with large and uncertain state spaces.

In the provided 2048 solver, MCTS is applied iteratively to determine the best move that maximizes the overall score based on the outcomes of simulated games.

In [3]:
def random_policy(game):
    game_copy = copy.deepcopy(game)
    while not game_copy.check_game_over():
        game_copy.move(random.randint(0, 3))
    return game_copy.score, max(max(row) for row in game_copy.grid), sum(sum(row) for row in game_copy.grid)

Sum = 0
Max = 0
Max_game = None
probs = [ 0 for _ in range(14)]

NUMBER_OF_GAMES = 100
NUMBER_OF_ITERATIONS = 75


# AI Solver using Monte Carlo Tree Search (MCTS) for 2048

## Overview

The AI agent for 2048 is implemented using the Monte Carlo Tree Search algorithm. This solver aims to make informed decisions about the best move in a given game state by simulating multiple random games and choosing the move that leads to the highest overall score.

## Components

### 1. Random Policy

- The `random_policy` function simulates random moves in a given game until a game-over state is reached.
- The score and the maximum tile value achieved in the simulated games are recorded.

### 2. MCTS (Monte Carlo Tree Search)

- The `mcts` function performs Monte Carlo Tree Search for each possible move (up, down, left, right).
- It utilizes the `random_policy` to simulate multiple games and accumulate scores for each move.
- The move with the highest accumulated score is selected as the best move.

### 3. Monte Carlo Simulation

- The `monte_carlo_simulation` function applies the MCTS algorithm iteratively until a game-over state is reached.
- It prints the final state of the game, the total score, and the maximum tile value achieved.

## Notes

- Adjust the number of iterations in the `random_policy` function based on computational resources and desired accuracy.
- The code assumes the existence of the `Game` class with the `move`, `check_game_over`, and other relevant methods.

In [ ]:

def mcts(initial_game, iterations):
    urdl_score = [0, 0, 0, 0]  # up, right, down, left

    for move in range(4):
        game_copy = copy.deepcopy(initial_game)
        result, grid = game_copy.move(move)

        if result == 2:
            continue

        # try random policy for 100 games
        for i in range(iterations):
            output = random_policy(game_copy)

            urdl_score[move] += output[0]


    best_move_by_score = urdl_score.index(max(urdl_score))

    initial_game.move(best_move_by_score)


def monte_carlo_simulation(initial_game, iterations):
    game = copy.deepcopy(initial_game)
    run = 1
    while not game.check_game_over():
        mcts(game, iterations)
        run += 1

    print("\n\nFinal State:\n", str(game))
    print("Score: " + str(game.score))
    print("Max Tile: " + str(max(max(row) for row in game.grid)))

    return game.score, max(max(row) for row in game.grid), game.grid
    
init_time = time.time()
for i in range(NUMBER_OF_GAMES):
    print("\n\nGame " + str(i + 1) + ":")
    game = Game(gui=False)
    (score, max_tile, grid) = monte_carlo_simulation(game, NUMBER_OF_ITERATIONS)

    power2 = math.log(max_tile, 2)

    for i in range(14):
        if int(power2) >= i:
            probs[i] += 1

    Sum += score
    Max = max(Max, score)

    Max_game = grid if score == Max else Max_game

print("\nTime taken (s): " + str(time.time() - init_time))
print("\nTime taken (min): " + str((time.time() - init_time)/60))
print("\nNumber of games: " + str(NUMBER_OF_GAMES))
print("\nNumber of rollouts for mcts: " + str(NUMBER_OF_ITERATIONS))
print("\nAverage Time per Game (s): " + str((time.time() - init_time)/NUMBER_OF_GAMES) + "\n")
print("\nAverage Time per Game (min): " + str((time.time() - init_time)/NUMBER_OF_GAMES/60) + "\n")

print("\nAverage Score: " + str(Sum / NUMBER_OF_GAMES))
print("\nMax Score: " + str(Max))

for i in range(14):
    if i != 13:
        print("\nProbability of getting " + str(2 ** i) + ": " + str(probs[i] / NUMBER_OF_GAMES))
    else:
        print("\nProbability of getting " + str(2 ** i) + " or more: " + str(probs[i] / NUMBER_OF_GAMES))

print("\n Max Game: \n", str(Max_game))




Game 1:


Final State:
 512 4 2 4
4 32 8 32
128 64 256 4
4 8 32 2
Score: 7216
Max Tile: 512


Game 2:


Final State:
 64 4 2 4
8 512 8 2
32 16 128 8
2 4 2 1024
Score: 14288
Max Tile: 1024


Game 3:


Final State:
 8 4 2048 4
2 32 64 128
4 16 512 4
2 256 8 16
Score: 27052
Max Tile: 2048


Game 4:


Final State:
 2 16 8 4
32 2048 64 32
128 1024 256 8
2 8 512 4
Score: 36236
Max Tile: 2048


Game 5:


Final State:
 2 256 16 8
32 64 32 2048
512 128 4 2
8 4 2 4
Score: 27188
Max Tile: 2048


Game 6:


Final State:
 8 512 8 4
256 128 16 2
2 32 4 1024
16 4 2 64
Score: 16092
Max Tile: 1024


Game 7:


Final State:
 2 4 2 16
8 128 64 1024
16 256 512 32
4 32 4 2
Score: 16212
Max Tile: 1024


Game 8:


Final State:
 4 2 8 512
1024 64 256 8
4 128 64 2
2 16 2 4
Score: 16212
Max Tile: 1024


Game 9:


Final State:
 4 2 4 8
2 16 32 4
64 8 128 4096
2048 2 4 2
Score: 65656
Max Tile: 4096


Game 10:


Final State:
 4 256 4 2
2048 128 512 4
8 32 64 16
2 4 16 2
Score: 27280
Max Tile: 2048


Game 11:


Fin

# Stats

| NUMBER OF TRIALS | ROLLOUTS PER MOVE | 2048 PROBABILITY | 4096 PROBABILITY | TOTAL TIME TAKEN            | AVERAGE TIME TAKEN PER GAME |
| ---------------- | ----------------- | ---------------- | ---------------- | --------------------------- | --------------------------- |
| 100              | 50                | 0.5              | 0.05             | 2 hour 1 min 2 second       | 1 minute 12 seconds         |
| 100              | 100               | 0.72             | 0.05             | 6 hours 30 mins 2 seconds   | 3 mins 54 seconds           |
| 100              | 200               | 0.81             | 0.1              | 22 hours 55 mins 26 seconds | 13 mins 45 seconds           |


## Heuristic Explanation

### Monte Carlo Tree Search with Max Score Selection (mctsms)

**Heuristic: Maximizing Score Through Random Simulations**

**Objective:**
The goal of this heuristic is to maximize the score in the game by selecting the move that yields the highest score in random simulations.

**Process:**

1. **Random Simulations:** For each valid move, a series of random simulations are conducted using a random policy (`random_policy`). 

2. **Score Recording:** The maximum score obtained from the random simulations is recorded for each move, forming a list of scores corresponding to each possible move.

3. **Move Selection:** The move with the highest estimated score is selected, and the initial game state is updated accordingly.

**Outcome:**
This heuristic is expected to yield a higher score than the other heuristics, as it focuses on maximizing the score through random simulations. But this didn't happen in our case. The reason for this is that the random policy used in the simulations is not optimized for maximizing the score. It is optimized for achieving the highest tile value possible. This is because the score is calculated based on the values of merged tiles, and the highest tile value is achieved by merging tiles of the same value. Therefore, the random policy is biased towards merging tiles of the same value, which may not necessarily lead to the highest score.

In [14]:

def mctsms(initial_game, iterations):
    urdl_score = [0, 0, 0, 0]  # up, right, down, left

    for move in range(4):
        game_copy = copy.deepcopy(initial_game)
        result, grid = game_copy.move(move)

        if result == 2:
            continue

        # try random policy for 100 games
        for i in range(iterations):
            output = random_policy(game_copy)

            urdl_score[move] = max(urdl_score[move], output[0])


    best_move_by_score = urdl_score.index(max(urdl_score))

    initial_game.move(best_move_by_score)


def monte_carlo_simulation_with_max_score(initial_game, iterations):
    game = copy.deepcopy(initial_game)
    run = 1
    while not game.check_game_over():
        mctsms(game, iterations)
        run += 1

    print("\n\nFinal State:\n", str(game))
    print("Score: " + str(game.score))
    print("Max Tile: " + str(max(max(row) for row in game.grid)))

    return game.score, max(max(row) for row in game.grid), game.grid



Sum = 0
Max = 0
Max_game = None
probs = [ 0 for _ in range(14)]

init_time = time.time()
for i in range(NUMBER_OF_GAMES):
    print("\n\nGame " + str(i + 1) + ":")
    game = Game(gui=False)
    (score, max_tile, grid) = monte_carlo_simulation_with_max_score(game, NUMBER_OF_ITERATIONS)

    power2 = math.log(max_tile, 2)

    for i in range(14):
        if int(power2) >= i:
            probs[i] += 1

    Sum += score
    Max = max(Max, score)

    Max_game = grid if score == Max else Max_game

print("\nTime taken (s): " + str(time.time() - init_time))
print("\nTime taken (min): " + str((time.time() - init_time)/60))
print("\nNumber of games: " + str(NUMBER_OF_GAMES))
print("\nNumber of rollouts for mcts: " + str(NUMBER_OF_ITERATIONS))
print("\nAverage Time per Game (s): " + str((time.time() - init_time)/NUMBER_OF_GAMES) + "\n")
print("\nAverage Time per Game (min): " + str((time.time() - init_time)/NUMBER_OF_GAMES/60) + "\n")

print("\nAverage Score: " + str(Sum / NUMBER_OF_GAMES))
print("\nMax Score: " + str(Max))

for i in range(14):
    if i != 13:
        print("\nProbability of getting " + str(2 ** i) + ": " + str(probs[i] / NUMBER_OF_GAMES))
    else:
        print("\nProbability of getting " + str(2 ** i) + " or more: " + str(probs[i] / NUMBER_OF_GAMES))

print("\n Max Game: \n", str(Max_game))




Game 1:


Final State:
 4 2 8 4
16 32 64 2
4 8 128 32
2 1024 16 4
Score: 10428
Max Tile: 1024


Game 2:


Final State:
 4 8 32 4
2 128 16 2
4 16 64 8
256 2 16 4
Score: 3084
Max Tile: 256


Game 3:


Final State:
 8 32 4 2
512 64 8 256
4 32 128 2
2 8 16 4
Score: 7104
Max Tile: 512


Game 4:


Final State:
 2 32 2 4
128 512 32 8
2 16 8 2
4 2 32 4
Score: 5192
Max Tile: 512


Game 5:


Final State:
 4 16 4 2
8 256 128 4
32 16 4 16
2 4 8 4
Score: 2800
Max Tile: 256


Game 6:


Final State:
 2 4 8 2
64 32 16 8
32 4 128 16
8 256 4 2
Score: 3196
Max Tile: 256


Game 7:


Final State:
 4 2 16 2
2 4 32 8
64 128 64 4
2 256 2 16
Score: 3320
Max Tile: 256


Game 8:


Final State:
 2 32 8 4
4 8 128 8
8 64 32 4
32 2 8 2
Score: 1496
Max Tile: 128


Game 9:


Final State:
 2 8 4 2
8 4 32 16
4 32 16 4
2 16 8 2
Score: 432
Max Tile: 32


Game 10:


Final State:
 2 128 2 4
16 32 16 2
8 256 64 16
2 4 16 4
Score: 3112
Max Tile: 256


Game 11:


Final State:
 2 4 32 4
32 8 256 32
4 16 64 4
2 8 16 2
Score: 2

## Heuristic Explanation

### Monte Carlo Tree Search with Mergeability Policy (mctsmg)

**Heuristic: Maximizing Mergeability to Encourage Tile Merging**

**Objective:**
The objective of this heuristic is to encourage tile merging during the game by considering the mergeability of tiles as a key factor. The heuristic aims to select moves that increase the likelihood of merging tiles during random simulations.

**Process:**

1. **Mergeability Policy:** The mergeability of each tile in the game is evaluated by examining its neighboring tiles. For each non-empty tile, the right and down neighbors are checked. If a neighboring tile has the same value, the mergeability score is increased.

2. **Random Simulations:** For each of the four possible moves (up, right, down, left), a copy of the initial game state is created, and the move is executed.

3. **Score Recording:** The mergeability policy score is added to the heuristic score (`urdl_score`) for each valid move during random simulations.

4. **Move Selection:** The move with the highest average mergeability score is selected, and the initial game state is updated accordingly.


In [15]:

def mergeability_policy(game):
    mergeability_score = 0

    for i in range(4):
        for j in range(4):
            current_tile = game.grid[i][j]
            if current_tile != 0:
                # Check right neighbor
                if j < 4 - 1 and game.grid[i][j + 1] == current_tile:
                    mergeability_score += 1
                # Check down neighbor
                if i < 4 - 1 and game.grid[i + 1][j] == current_tile:
                    mergeability_score += 1

    return mergeability_score

def mctsmg(initial_game, iterations):
    urdl_score = [0, 0, 0, 0]  # up, right, down, left

    for move in range(4):
        game_copy = copy.deepcopy(initial_game)
        result, grid = game_copy.move(move)

        if result == 2:
            continue

        # try random policy for 100 games
        for i in range(iterations):
            output = random_policy(game_copy)

            urdl_score[move] += mergeability_policy(game_copy) # this or we will consider max value only


    best_move_by_score = urdl_score.index(max(urdl_score))

    initial_game.move(best_move_by_score)


def monte_carlo_simulation_with_mergability(initial_game, iterations):
    game = copy.deepcopy(initial_game)
    run = 1
    while not game.check_game_over():
        mctsmg(game, iterations)
        run += 1

    print("\n\nFinal State:\n", str(game))
    print("Score: " + str(game.score))
    print("Max Tile: " + str(max(max(row) for row in game.grid)))

    return game.score, max(max(row) for row in game.grid), game.grid



Sum = 0
Max = 0
Max_game = None
probs = [ 0 for _ in range(14)]

init_time = time.time()
for i in range(NUMBER_OF_GAMES):
    print("\n\nGame " + str(i + 1) + ":")
    game = Game(gui=False)
    (score, max_tile, grid) = monte_carlo_simulation_with_max_score(game, NUMBER_OF_ITERATIONS)

    power2 = math.log(max_tile, 2)

    for i in range(14):
        if int(power2) >= i:
            probs[i] += 1

    Sum += score
    Max = max(Max, score)

    Max_game = grid if score == Max else Max_game

print("\nTime taken (s): " + str(time.time() - init_time))
print("\nTime taken (min): " + str((time.time() - init_time)/60))
print("\nNumber of games: " + str(NUMBER_OF_GAMES))
print("\nNumber of rollouts for mcts: " + str(NUMBER_OF_ITERATIONS))
print("\nAverage Time per Game (s): " + str((time.time() - init_time)/NUMBER_OF_GAMES) + "\n")
print("\nAverage Time per Game (min): " + str((time.time() - init_time)/NUMBER_OF_GAMES/60) + "\n")

print("\nAverage Score: " + str(Sum / NUMBER_OF_GAMES))
print("\nMax Score: " + str(Max))

for i in range(14):
    if i != 13:
        print("\nProbability of getting " + str(2 ** i) + ": " + str(probs[i] / NUMBER_OF_GAMES))
    else:
        print("\nProbability of getting " + str(2 ** i) + " or more: " + str(probs[i] / NUMBER_OF_GAMES))

print("\n Max Game: \n", str(Max_game))




Game 1:


Final State:
 4 2 4 8
512 16 8 2
2 8 256 4
128 2 32 2
Score: 6716
Max Tile: 512


Game 2:


Final State:
 8 128 8 4
2 64 16 2
8 16 32 16
2 4 256 2
Score: 3116
Max Tile: 256


Game 3:


Final State:
 8 2 4 2
2 8 64 128
4 64 16 4
16 2 8 2
Score: 1488
Max Tile: 128


Game 4:


Final State:
 2 256 8 4
8 16 128 64
4 32 16 8
2 8 2 4
Score: 3092
Max Tile: 256


Game 5:


Final State:
 4 64 4 2
8 4 16 4
2 128 32 16
512 4 8 4
Score: 5292
Max Tile: 512


Game 6:


Final State:
 2 4 2 4
16 8 4 8
1024 256 16 32
16 4 256 2
Score: 12844
Max Tile: 1024


Game 7:


Final State:
 4 8 16 4
16 2 256 32
4 512 2 8
2 8 32 2
Score: 6156
Max Tile: 512


Game 8:


Final State:
 8 2 4 8
16 64 8 16
2 512 32 4
4 2 8 2
Score: 4572
Max Tile: 512


Game 9:


Final State:
 2 16 2 4
4 8 4 64
32 128 8 2
4 2 1024 4
Score: 10328
Max Tile: 1024


Game 10:


Final State:
 2 4 8 4
16 32 64 2
4 16 128 8
2 8 16 4
Score: 1372
Max Tile: 128


Game 11:


Final State:
 512 8 16 2
2 4 128 8
8 32 4 2
4 8 2 4
Score: 4988

## Heuristic Explanation

### Monte Carlo Tree Search with Monotonicity Policy (mctsmt)

**Heuristic: Encouraging Monotonicity in Tile Values**

**Objective:**
The goal of this heuristic is to encourage the monotonicity of tile values in the game grid. Monotonicity implies that the tile values increase or decrease consistently in a particular row or column. The heuristic aims to select moves that increase the overall monotonicity of the grid.

**Process:**

1. **Monotonicity Policy:** The monotonicity of each row in the game grid is evaluated by comparing adjacent tile values. If the tile values are in non-decreasing order from left to right, or non-increasing order from left to right, a monotonicity score is calculated. This is done for each row.

2. **Random Simulations:** For each of the four possible moves (up, right, down, left), a copy of the initial game state is created, and the move is executed. 

3. **Score Recording:** The monotonicity policy score is added to the heuristic score (`urdl_score`) for each valid move during random simulations.

4. **Move Selection:** The move with the highest estimated monotonicity score is selected, and the initial game state is updated accordingly.


In [7]:

def monotonicity_policy(game):
    monotonicity_score = 0

    for i in range(4):
        for j in range(1, 4):
            if game.grid[i][j] >= game.grid[i][j - 1]:
                monotonicity_score += game.grid[i][j] - game.grid[i][j - 1]
            else:
                monotonicity_score += game.grid[i][j - 1] - game.grid[i][j]

    return monotonicity_score

def mctsmt(initial_game, iterations):
    urdl_score = [0, 0, 0, 0]  # up, right, down, left

    for move in range(4):
        game_copy = copy.deepcopy(initial_game)
        result, grid = game_copy.move(move)

        if result == 2:
            continue

        # try random policy for 100 games
        for i in range(iterations):
            output = random_policy(game_copy)

            urdl_score[move] += monotonicity_policy(game_copy) # this or we will consider max value only


    best_move_by_score = urdl_score.index(max(urdl_score))

    initial_game.move(best_move_by_score)


def monte_carlo_simulation_with_monotonicity(initial_game, iterations):
    game = copy.deepcopy(initial_game)
    run = 1
    while not game.check_game_over():
        mctsmt(game, iterations)
        run += 1

    print("\n\nFinal State:\n", str(game))
    print("Score: " + str(game.score))
    print("Max Tile: " + str(max(max(row) for row in game.grid)))

    return game.score, max(max(row) for row in game.grid), game.grid



Sum = 0
Max = 0
Max_game = None
probs = [ 0 for _ in range(14)]

init_time = time.time()
for i in range(NUMBER_OF_GAMES):
    print("\n\nGame " + str(i + 1) + ":")
    game = Game(gui=False)
    (score, max_tile, grid) = monte_carlo_simulation_with_monotonicity(game, NUMBER_OF_ITERATIONS)

    power2 = math.log(max_tile, 2)

    for i in range(14):
        if int(power2) >= i:
            probs[i] += 1

    Sum += score
    Max = max(Max, score)

    Max_game = grid if score == Max else Max_game

print("\nTime taken (s): " + str(time.time() - init_time))
print("\nTime taken (min): " + str((time.time() - init_time)/60))
print("\nNumber of games: " + str(NUMBER_OF_GAMES))
print("\nNumber of rollouts for mcts: " + str(NUMBER_OF_ITERATIONS))
print("\nAverage Time per Game (s): " + str((time.time() - init_time)/NUMBER_OF_GAMES) + "\n")
print("\nAverage Time per Game (min): " + str((time.time() - init_time)/NUMBER_OF_GAMES/60) + "\n")

print("\nAverage Score: " + str(Sum / NUMBER_OF_GAMES))
print("\nMax Score: " + str(Max))


for i in range(14):
    if i != 13:
        print("\nProbability of getting " + str(2 ** i) + ": " + str(probs[i] / NUMBER_OF_GAMES))
    else:
        print("\nProbability of getting " + str(2 ** i) + " or more: " + str(probs[i] / NUMBER_OF_GAMES))

print("\n Max Game: \n", str(Max_game))



Game 1:


Final State:
 16 2 8 2
8 32 2 16
4 16 8 2
2 8 2 8
Score: 332
Max Tile: 32


Game 2:


Final State:
 2 16 2 4
8 4 64 2
4 16 4 8
2 4 8 2
Score: 448
Max Tile: 64


Game 3:


Final State:
 2 32 4 16
16 2 64 4
2 4 16 2
4 8 2 4
Score: 608
Max Tile: 64


Game 4:


Final State:
 2 8 2 8
16 4 8 2
8 2 16 4
2 4 8 2
Score: 172
Max Tile: 16


Game 5:


Final State:
 2 4 2 16
8 2 64 8
4 64 8 4
2 8 4 2
Score: 728
Max Tile: 64


Game 6:


Final State:
 2 16 4 8
4 64 8 2
16 4 32 8
4 2 4 2
Score: 584
Max Tile: 64


Game 7:


Final State:
 2 32 2 8
16 8 16 2
4 2 32 4
2 4 8 2
Score: 404
Max Tile: 32


Game 8:


Final State:
 2 4 2 16
8 2 64 8
4 8 32 4
2 16 4 2
Score: 568
Max Tile: 64


Game 9:


Final State:
 2 4 2 4
4 32 8 2
2 16 2 4
16 2 16 2
Score: 284
Max Tile: 32


Game 10:


Final State:
 2 4 16 2
4 16 32 4
8 2 16 8
2 8 2 16
Score: 368
Max Tile: 32


Game 11:


Final State:
 2 32 2 16
8 2 16 4
4 16 8 2
2 4 16 4
Score: 344
Max Tile: 32


Game 12:


Final State:
 2 8 16 2
16 2 4 8
4 32 2 4

## Heuristic Explanation

### Monte Carlo Tree Search with Tile Sum Selection (mctsts)

**Heuristic: Maximizing Tile Sum Through Random Simulations**

**Objective:**
The objective of this heuristic is to maximize the sum of tile values in the game grid. It aims to achieve this by selecting moves that lead to a higher total sum of tile values during random simulations.

**Process:**

1. **Random Simulations:** For each of the four possible moves (up, right, down, left), a copy of the initial game state is created, and the move is executed.

2. **Tile Sum Recording:** The sum of tile values in the grid is recorded for each move during random simulations. This forms a list of sums corresponding to each possible move.

3. **Move Selection:** The move with the highest estimated sum of tile values is selected, and the initial game state is updated accordingly.

**Outcome:**
This heuristic prioritizes moves that lead to a higher total sum of tile values in the game grid. The intention is to create a grid configuration where the sum of tile values is maximized, potentially resulting in a higher score.

In [4]:

def mctsts(initial_game, iterations):
    urdl_score = [0, 0, 0, 0]  # up, right, down, left

    for move in range(4):
        game_copy = copy.deepcopy(initial_game)
        result, grid = game_copy.move(move)

        if result == 2:
            continue

        # try random policy for 100 games
        for i in range(iterations):
            output = random_policy(game_copy)

            urdl_score[move] += (output[2] + output[0])


    best_move_by_score = urdl_score.index(max(urdl_score))

    initial_game.move(best_move_by_score)


def monte_carlo_simulation_with_tile_sum(initial_game, iterations):
    game = copy.deepcopy(initial_game)
    run = 1
    while not game.check_game_over():
        mctsts(game, iterations)
        run += 1

    print("\n\nFinal State:\n", str(game))
    print("Score: " + str(game.score))
    print("Max Tile: " + str(max(max(row) for row in game.grid)))

    return game.score, max(max(row) for row in game.grid), game.grid



Sum = 0
Max = 0
Max_game = None
probs = [ 0 for _ in range(14)]

init_time = time.time()
for i in range(NUMBER_OF_GAMES):
    print("\n\nGame " + str(i + 1) + ":")
    game = Game(gui=False)
    (score, max_tile, grid) = monte_carlo_simulation_with_tile_sum(game, NUMBER_OF_ITERATIONS)

    power2 = math.log(max_tile, 2)

    for i in range(14):
        if int(power2) >= i:
            probs[i] += 1

    Sum += score
    Max = max(Max, score)

    Max_game = grid if score == Max else Max_game

print("\nTime taken (s): " + str(time.time() - init_time))
print("\nTime taken (min): " + str((time.time() - init_time)/60))
print("\nNumber of games: " + str(NUMBER_OF_GAMES))
print("\nNumber of rollouts for mcts: " + str(NUMBER_OF_ITERATIONS))
print("\nAverage Time per Game (s): " + str((time.time() - init_time)/NUMBER_OF_GAMES) + "\n")
print("\nAverage Time per Game (min): " + str((time.time() - init_time)/NUMBER_OF_GAMES/60) + "\n")

print("\nAverage Score: " + str(Sum / NUMBER_OF_GAMES))
print("\nMax Score: " + str(Max))

for i in range(14):
    if i != 13:
        print("\nProbability of getting " + str(2 ** i) + ": " + str(probs[i] / NUMBER_OF_GAMES))
    else:
        print("\nProbability of getting " + str(2 ** i) + " or more: " + str(probs[i] / NUMBER_OF_GAMES))

print("\n Max Game: \n", str(Max_game))




Game 1:


Final State:
 32 2 32 2
8 512 128 4
4 2048 256 8
2 128 16 4
Score: 27680
Max Tile: 2048


Game 2:


Final State:
 2 8 4 16
64 128 32 512
16 256 8 4
2 2048 16 2
Score: 27248
Max Tile: 2048


Game 3:


Final State:
 2 8 16 4
4 64 32 1024
128 256 16 4
2 32 2 512
Score: 16200
Max Tile: 1024


Game 4:


Final State:
 4 2048 256 2
1024 32 128 4
4 512 32 16
2 4 64 2
Score: 36200
Max Tile: 2048


Game 5:


Final State:
 2 4 8 4
16 256 64 16
4 2048 128 32
32 512 2 8
Score: 27208
Max Tile: 2048


Game 6:


Final State:
 2 4 16 4
8 512 128 2
16 32 1024 64
4 256 2 4
Score: 16052
Max Tile: 1024


Game 7:


Final State:
 4 256 16 8
32 1024 64 32
4 16 128 4
2 4 512 2
Score: 16148
Max Tile: 1024


Game 8:


Final State:
 2 16 4 8
256 128 32 1024
16 512 64 4
2 4 32 8
Score: 16184
Max Tile: 1024


Game 9:


Final State:
 2 16 32 4
8 1024 256 64
2048 32 512 4
2 16 4 2
Score: 35628
Max Tile: 2048


Game 10:


| Number of Trials | Number of Rollouts | 1024 Probability | 2048 Probability | 4096 Probability | Average Time Taken per Game | Average Score |
| ---------------- | ------------------ | ---------------- | ---------------- | ---------------- | --------------------------- | ------------- |
| 25               | 50                 | 0.96             | 0.72             | 0.04             | 115 seconds                 | 26407         |
| 100              | 50                 | 0.91             | 0.54             | 0.03             | 108 seconds                 | 24034         |


## Heuristic Explanation

### Monte Carlo Tree Search with Top-Right Corner Bias (mctstrc)

**Heuristic: Biased Exploration for Top-Right Corner**

**Objective:**
This heuristic introduces a bias during the exploration phase of Monte Carlo Tree Search (MCTS) to favor moves that lead to the top-right corner of the game grid. The bias is applied to maximize the score by directing the search toward a particular corner.

**Process:**

1. **Random Simulations:** For each of the four possible moves (up, right, down, left), a copy of the initial game state is created, and the move is executed.

2. **Score Recording:** The scores obtained from random simulations are recorded for each move. The bias is applied by adjusting the scores for up and left moves to encourage exploration towards the top-right corner.

3. **Move Selection:** The move with the highest adjusted score is selected, and the initial game state is updated accordingly.

**Outcome:**
The heuristic aims to encourage the AI to explore and favor moves that direct the game tiles towards the top-right corner. By biasing towards this specific direction, the AI intends to create a game configuration that maximizes the score. The adjustment factors for up and left moves, 1.2 and 1.15 respectively, introduce a slight bias towards these directions, potentially impacting the AI's decision-making process.

In [14]:

def mctstrc(initial_game, iterations):
    urdl_score = [0, 0, 0, 0]  # up, right, down, left

    for move in range(4):
        game_copy = copy.deepcopy(initial_game)
        result, grid = game_copy.move(move)

        if result == 2:
            continue

        # try random policy for 100 games
        for i in range(iterations):
            output = random_policy(game_copy)

            urdl_score[move] += output[0]
    
    urdl_score[0] *= 1.2
    urdl_score[1] *= 1.15
    urdl_score[2] *= 1.15
    urdl_score[3] *= 1.2


    best_move_by_score = urdl_score.index(max(urdl_score))

    initial_game.move(best_move_by_score)


def monte_carlo_simulation_with_top_right_corner(initial_game, iterations):
    game = copy.deepcopy(initial_game)
    run = 1
    while not game.check_game_over():
        mctstrc(game, iterations)
        run += 1

    print("\n\nFinal State:\n", str(game))
    print("Score: " + str(game.score))
    print("Max Tile: " + str(max(max(row) for row in game.grid)))

    return game.score, max(max(row) for row in game.grid), game.grid



Sum = 0
Max = 0
Max_game = None
probs = [ 0 for _ in range(14)]

init_time = time.time()
for i in range(NUMBER_OF_GAMES):
    print("\n\nGame " + str(i + 1) + ":")
    game = Game(gui=False)
    (score, max_tile, grid) = monte_carlo_simulation_with_top_right_corner(game, NUMBER_OF_ITERATIONS)

    power2 = math.log(max_tile, 2)

    for i in range(14):
        if int(power2) >= i:
            probs[i] += 1

    Sum += score
    Max = max(Max, score)

    Max_game = grid if score == Max else Max_game

print("\nTime taken (s): " + str(time.time() - init_time))
print("\nTime taken (min): " + str((time.time() - init_time)/60))
print("\nNumber of games: " + str(NUMBER_OF_GAMES))
print("\nNumber of rollouts for mcts: " + str(NUMBER_OF_ITERATIONS))
print("\nAverage Time per Game (s): " + str((time.time() - init_time)/NUMBER_OF_GAMES) + "\n")
print("\nAverage Time per Game (min): " + str((time.time() - init_time)/NUMBER_OF_GAMES/60) + "\n")

print("\nAverage Score: " + str(Sum / NUMBER_OF_GAMES))
print("\nMax Score: " + str(Max))

for i in range(14):
    if i != 13:
        print("\nProbability of getting " + str(2 ** i) + ": " + str(probs[i] / NUMBER_OF_GAMES))
    else:
        print("\nProbability of getting " + str(2 ** i) + " or more: " + str(probs[i] / NUMBER_OF_GAMES))

print("\n Max Game: \n", str(Max_game))




Game 1:


Final State:
 32 4 32 8
1024 64 16 4
4 16 8 2
16 8 2 4
Score: 9768
Max Tile: 1024


Game 2:


Final State:
 2 16 4 2
1024 4 32 16
4 2 8 4
16 8 4 2
Score: 9364
Max Tile: 1024


Game 3:


Final State:
 32 4 64 2
512 128 16 8
8 64 8 4
4 32 4 2
Score: 5728
Max Tile: 512


Game 4:


Final State:
 8 2 8 2
64 8 16 8
32 1024 8 4
2 8 4 2
Score: 9612
Max Tile: 1024


Game 5:


Final State:
 32 4 64 2
4 128 512 8
16 64 32 4
4 8 4 2
Score: 5704
Max Tile: 512


Game 6:


Final State:
 8 32 8 2
1024 64 32 8
2 16 8 4
16 8 4 2
Score: 9792
Max Tile: 1024


Game 7:


Final State:
 8 4 32 2
2 512 128 8
128 32 8 4
32 8 4 2
Score: 5952
Max Tile: 512


Game 8:


Final State:
 64 512 64 2
32 256 32 16
4 64 16 4
16 8 4 2
Score: 7092
Max Tile: 512


Game 9:


Final State:
 256 16 2 4
64 8 4 2
4 128 32 4
2 8 4 2
Score: 2996
Max Tile: 256


Game 10:


Final State:
 64 4 2 4
16 32 128 2
2 512 16 4
16 2 8 2
Score: 5280
Max Tile: 512


Game 11:


Final State:
 8 64 8 2
256 512 64 8
64 8 4 2
2 4 2 4
Scor

50 Rollouts and 25 games

| Heuristic                                  | 1024 Probability | 2048 Probability | 4096 probability | Average Score | Max Score | Average Time |
| ------------------------------------------ | ---------------- | ---------------- | ---------------- | ------------- | --------- | ------------ |
| Maximum Monotonicity                       | 0                | 0                | 0                | 512           | 960       | 33 seconds   |
| Maximum Score per Round                    | 0.12             | 0                | 0                | 5312          | 15164     | 61 seconds   |
| Average Mergability                        | 0.24             | 0                | 0                | 5974          | 14044     | 52 seconds   |
| Sum of Scores with priority to up and left | 0.24             | 0           | 0                | 6654         | 11480     | 64 seconds   |
| Sum of Scores per Round                    | 0.92             | 0.56             | 0.04             | 24798         | 65656     | 200 seconds  |
| Sum of Scores and Tile Values per Round    | 0.96             | 0.72             | 0.04             | 26407         | 51136     | 115 seconds  |
